In [15]:
import pandas as pd
import numpy as np

In [16]:
#import dataset
df = pd.read_csv(r'C:\Users\Ahmed\Desktop\sonnen\measurements_coding_challenge.csv',delimiter=';')
df

,timestamp,serial,grid_purchase,grid_feedin,direct_consumption,date
0,2023-05-22T15:40:00.000Z,1711356005,673,0,NaN,2023-05-22
1,2023-05-22T15:54:00.000Z,2105167400,NaN,36,NaN,2023-05-22
2,2023-05-22T16:14:00.000Z,1083091999,7012,0,NaN,2023-05-22
3,2023-05-22T04:57:00.000Z,970568993,0,2045,NaN,2023-05-22
4,2023-05-22T16:30:00.000Z,1559834999,27,0,NaN,2023-05-22
...,...,...,...,...,...,...
15823,2023-05-22T17:15:00.000Z,2105167400,NaN,0,NaN,2023-05-22
15824,2023-05-22T18:58:00.000Z,517262015,29,0,NaN,2023-05-22
15825,2023-05-22T15:04:00.000Z,1135739597,6918,0,NaN,2023-05-22
15826,2023-05-22T18:05:00.000Z,970568993,460,0,NaN,2023-05-22


In [17]:
df.isnull().sum()

timestamp                 0
serial                    0
grid_purchase          1438
grid_feedin               0
direct_consumption    15826
date                      0
dtype: int64

In [18]:
#delete column direct_consumption
df = df.drop('direct_consumption',axis=1)
df.head()

,timestamp,serial,grid_purchase,grid_feedin,date
0,2023-05-22T15:40:00.000Z,1711356005,673,0,2023-05-22
1,2023-05-22T15:54:00.000Z,2105167400,NaN,36,2023-05-22
2,2023-05-22T16:14:00.000Z,1083091999,7012,0,2023-05-22
3,2023-05-22T04:57:00.000Z,970568993,0,2045,2023-05-22
4,2023-05-22T16:30:00.000Z,1559834999,27,0,2023-05-22


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15828 entries, 0 to 15827
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   timestamp      15828 non-null  object
 1   serial         15828 non-null  int64 
 2   grid_purchase  14390 non-null  object
 3   grid_feedin    15828 non-null  object
 4   date           15828 non-null  object
dtypes: int64(1), object(4)
memory usage: 618.4+ KB


In [20]:
#Replace missing values with appropriate default values
df.fillna({'grid_purchase': 0, 'grid_feedin': 0}, inplace=True)
df.isnull().sum()

timestamp        0
serial           0
grid_purchase    0
grid_feedin      0
date             0
dtype: int64

In [ ]:
#Convert columns to their appropriate data types
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15828 entries, 0 to 15827
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp      15828 non-null  datetime64[ns, UTC]
 1   serial         15828 non-null  int64              
 2   grid_purchase  15828 non-null  object             
 3   grid_feedin    15828 non-null  object             
 4   date           15828 non-null  datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int64(1), object(2)
memory usage: 618.4+ KB


In [22]:
df['grid_purchase'] = pd.to_numeric(df['grid_purchase'], errors='coerce').fillna(0).astype(int)
df['grid_feedin'] = pd.to_numeric(df['grid_feedin'], errors='coerce').fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15828 entries, 0 to 15827
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp      15828 non-null  datetime64[ns, UTC]
 1   serial         15828 non-null  int64              
 2   grid_purchase  15828 non-null  int32              
 3   grid_feedin    15828 non-null  int32              
 4   date           15828 non-null  datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int32(2), int64(1)
memory usage: 494.8 KB


In [23]:
#Remove duplicates and corrupt values from the dataset.
df.drop_duplicates(inplace=True)

In [24]:
#calculate the total grid_purchase and grid_feedin over all batteries for each hour of the day
df['hour'] = df['timestamp'].dt.hour
hourly_agg = df.groupby('hour')[['grid_purchase', 'grid_feedin']].sum().reset_index()
hourly_agg

,hour,grid_purchase,grid_feedin
0,0,2125602,50482
1,1,1420502,71651
2,2,1632835,118036
3,3,770287,343602
4,4,216737,572127
5,5,167730,127197
6,6,262075,11423
7,7,873026,5184
8,8,1600957,3203
9,9,1236858,3477


In [25]:
#Add a column to your dataframe that indicates the hour with the highest grid_feedin of the day
df['highest_grid_feedin_hour_of_day'] = df['hour'] == df.groupby('date')['grid_feedin'].transform('idxmax')
df

,timestamp,serial,grid_purchase,grid_feedin,date,hour,highest_grid_feedin_hour_of_day
0,2023-05-22 15:40:00+00:00,1711356005,673,0,2023-05-22,15,False
1,2023-05-22 15:54:00+00:00,2105167400,0,36,2023-05-22,15,False
2,2023-05-22 16:14:00+00:00,1083091999,7012,0,2023-05-22,16,False
3,2023-05-22 04:57:00+00:00,970568993,0,2045,2023-05-22,4,False
4,2023-05-22 16:30:00+00:00,1559834999,27,0,2023-05-22,16,False
...,...,...,...,...,...,...,...
15823,2023-05-22 17:15:00+00:00,2105167400,0,0,2023-05-22,17,False
15824,2023-05-22 18:58:00+00:00,517262015,29,0,2023-05-22,18,False
15825,2023-05-22 15:04:00+00:00,1135739597,6918,0,2023-05-22,15,False
15826,2023-05-22 18:05:00+00:00,970568993,460,0,2023-05-22,18,False


In [26]:
#save the output
df.to_csv(r'C:\Users\Ahmed\Desktop\sonnen\transformed_data.csv',index=False)